# Overview
This notebook is designed to demonstrate how the attached SQL database was constructed. Weekly Shonen Jump is a weekly manga (comic book) magazine published in Japan which includes roughly 20 regularly serialized series per issue. The database `shonen_jump.sqlite3` records series published in the magazine and tracks when series perform well (and are placed higher in the magazine's table of contents), receive color pages, or grace the cover of the magazine.

In [1]:
import sqlite3
connection = sqlite3.connect("shonen_jump.sqlite3")
cursor=connection.cursor()

# We will enable foreign keys
cursor.execute("PRAGMA foreign_keys = ON")

## statuses Table

First we will create a table to hold the possible statuses of a manga: "Ongoing", "Complete", "Hiatus".

In [2]:
cursor.execute("CREATE TABLE IF NOT EXISTS statuses(status TEXT PRIMARY KEY)")
cursor.execute("""
               INSERT OR IGNORE INTO statuses VALUES
               ('Ongoing'),
               ('Complete'),
               ('Hiatus'),
               ('Transferred')""")
connection.commit()

## genres Table

Similarly we will create a table of major genres.

In [3]:
cursor.execute("CREATE TABLE IF NOT EXISTS genres(genre TEXT PRIMARY KEY)")
cursor.execute("""
               INSERT OR IGNORE INTO genres VALUES
               ('Battle'),
               ('Sports'),
               ('Romance'),
               ('Comedy'),
               ('Other')""")
connection.commit()

## series Table

Next we will create a table to record which series are running (or have run) in the magazine.

In [4]:
cursor.execute("""CREATE TABLE IF NOT EXISTS series(
               title TEXT PRIMARY KEY,
               writer TEXT,
               artist TEXT,
               total_chapters INTEGER CHECK (total_chapters > 0),
               genre TEXT,
               status TEXT,
               FOREIGN KEY(status) REFERENCES statuses(status),
               FOREIGN KEY(genre) REFERENCES genres(genre)
               )""")

### Example Data Entry

To add multiple series at once to this new table we will use the execute many command. First we collect information on the series.

In [5]:
# Series will be added in the form (title, chapters, writer, artist, start, genre)

series_to_add=[
    ('One Piece', 1156,'Eiichiro Oda','Eiichiro Oda', 'Battle'),
    ('Kagurabachi',89, 'Takeru Hokazono','Takeru Hokazono', 'Battle'),
    ('Sakamoto Days',224, 'Yuto Suzuki','Yuto Suzuki', 'Battle'),
    ('Ichi the Witch',45, 'Osamu Nishi','Shiro Usazaki', 'Battle'),
    ('Kaedegami',7, 'Jun Harukawa','Jun Harukawa', 'Battle'),
    ('The Elusive Samurai',213,'Yusei Matsui','Yusei Matsui', 'Other'),
    ('Shinobi Undercover',44,'Ippon Takegushi','Santa Mitarashi', 'Battle'),
    ("Nue's Exorcist",109,'Kota Kawae','Kota Kawae', 'Battle'),
    ('Ultimate Exorcist Kiyoshi', 55,'Shoichi Usui','Shoichi Usui', 'Battle'),
    ('Hima-Ten!',53,'Genki Ono','Genki Ono', 'Romance'),
    ('WITCH WATCH',212,'Kenta Shinohara','Kenta Shinohara', 'Comedy'),
    ('Akane-banashi',169,'Yuki Suenaga','Takamasa Moue', 'Other'),
    ('Blue Box',206,'Kouji Miura','Kouji Miura', 'Romance'),
    ('Ekiden Bros',6,'Daiki Nono','Daiki Nono', 'Sports'),
    ('Harukaze Mound', 8, 'Togo Goto', 'Kento Matsuura',  'Sports'),
    ('Kill Blue', 112, 'Tadatoshi Fujimaki','Tadatoshi Fujimaki','Comedy'),
    ('Me & Roboco',244,'Shuhei Miyazaki','Shuhei Miyazaki','Comedy'),
    ('Nice Prison',15,'Tatsuya Suganuma','Tatsuya Suganuma','Comedy'),
    ('Otr of the Flame',13,'Yuki Kawaguchi','Yuki Kawaguchi','Battle'),
    ('Ping-Pong Peril',5,'Yoshiharu Kataoka','Yoshiharu Kataoka','Sports'),
    ('Jujutsu Kaisen Modulo',7,'Gege Akutami','Yuji Iwasaki','Battle'),
    ('Someone Hertz',6,'Ei Yamano','Ei Yamano','Romance'),
    ('Gonron Egg',3,'Shuhei Tanizaki','Shuhei Tanizaki','Battle'),
    ('The Mage Next Door', 2, 'Hideaki Nabe', 'Hideaki Nabe', 'Comedy'),
    ('Hero Girl and Demon Lord Call It Quits',1,'Matsuri Hatsubina','Matsuri Hatsubina','Romance')
]

In [6]:
cursor.executemany("""
               INSERT OR IGNORE INTO series (title, total_chapters, writer, artist, genre, status) 
                   VALUES (?,?,?,?,?,'Ongoing')""",series_to_add)
connection.commit()

## chapter_types Table

Now we will turn our attention to creating a table listing individual chapters appearing in the magazine. First we will list the kinds of chapters possible.

In [7]:
cursor.execute("CREATE TABLE IF NOT EXISTS chapter_types(type TEXT PRIMARY KEY DEFAULT 'Normal')")
cursor.execute("""
               INSERT OR IGNORE INTO chapter_types VALUES
               ('Normal'),
               ('Color'),
               ('Cover'),
               ('Absent')""")
connection.commit()

## chapters Table

In [8]:
cursor.execute("""CREATE TABLE IF NOT EXISTS chapters(
               series TEXT NOT NULL,
               release_date TEXT NOT NULL,
               toc_rank INTEGER,
               chapter INTEGER,
               type TEXT,
               placement INTEGER,
               Primary KEY (series, release_date),
               FOREIGN KEY(series) REFERENCES series(title)
               )""")
connection.commit()

### Creating Chapter Count Triggers

We also want the total chapter count in the series table to increment by 1 whenever a new chapter is published.  For this we will create two triggers for when the chapters table is acted upon.  One for insertions and one for updates.

In [9]:
cursor.execute("""
               CREATE TRIGGER IF NOT EXISTS update_chapter_on_insert INSERT ON chapters
               WHEN NEW.chapter > (SELECT total_chapters FROM series WHERE title = NEW.series)
               BEGIN
               UPDATE series SET total_chapters = NEW.chapter WHERE title = NEW.series;
               END;
""")

cursor.execute("""
               CREATE TRIGGER IF NOT EXISTS update_chapter_on_update UPDATE OF chapter ON chapters
               WHEN NEW.chapter > (SELECT total_chapters FROM series WHERE title = NEW.series)
               BEGIN
               UPDATE series SET total_chapters = NEW.chapter WHERE title = NEW.series;
               END;
""")
connection.commit()

We fill individual chapters in using the notebook `interactive_entry.ipynb` to gather table of contents information from the internet and add it to the chapters table of our SQLite3 file.

To fill in the chapters table we will use the `add_week` function created in the script `sj_db_functions.py` which simplifies the process of adding each week's worth of chapters. The function is repeated below for completeness. 

## batches Table

Manga in Weekly Shonen Jump are typically added and removed in batches. For instance, over the course of three successive weeks one series ends per week and then over the following three weeks a new series debuts each week. We will create a table to track when batches start and end, as well as counting how many series were concluded and started with the batch. The column start_date contains the date of the first cancellation in the batch and the column end_date contains the date of the last new series added.

In [10]:
cursor.execute("""CREATE TABLE IF NOT EXISTS batches(
               start_date TEXT NOT NULL,
               end_date TEXT NOT NULL,
               added INTEGER,
               completed INTEGER,
               Primary KEY (start_date, end_date)
               );""")
connection.commit()

## absences Table

In this table we will record weeks in which a series (during it's serialization) was on break. In `sj_db_functions.py` we will create functions to populate this table and handle longer hiatuses.

In [11]:
cursor.execute("""CREATE TABLE IF NOT EXISTS absences(
               series TEXT NOT NULL,
               issue_date TEXT NOT NULL,
               Primary KEY (series, issue_date),
               FOREIGN KEY(series) REFERENCES series(title)
               )""")
connection.commit()

## hiatuses Table

In this table we will record hiatuses, which we define as three or more consecutive absences. We record the series name, the start and end dates, and the length (in issues) of the hiatus.

In [12]:
cursor.execute("""CREATE TABLE IF NOT EXISTS hiatuses(
               series TEXT NOT NULL,
               start_date TEXT NOT NULL,
               end_date TEXT NOT NULL,
               length INTEGER,
               Primary KEY (series, start_date, end_date),
               FOREIGN KEY(series) REFERENCES series(title)
               )""")
connection.commit()

## Creating Views

### debuts

The first view we create will list only the debut chapter for each series.

In [13]:
cursor.execute("""CREATE VIEW IF NOT EXISTS debuts AS
                SELECT series, release_date, chapter FROM chapters
                WHERE chapter=1
                ORDER BY release_date DESC;""")
connection.commit()

### finales

Next we create a view that only displays the final chapter published in Weekly Shonen Jump.

In [14]:
cursor.execute("""CREATE VIEW IF NOT EXISTS finales AS
               SELECT series, 
               release_date,
                chapter
                FROM chapters
                JOIN series ON series.title=chapters.series AND 
                chapters.chapter=series.total_chapters AND
                (series.status='Complete' OR series.status='Transferred')
                GROUP BY chapters.series
                ORDER BY release_date DESC;""")
connection.commit()

### batch_locator

Now we combine these views to be able to easily view batches.

In [15]:
cursor.execute("""CREATE VIEW IF NOT EXISTS batch_locator AS
                SELECT *, 'Debut' AS debut_or_finale from debuts 
                UNION SELECT *, 'Finale' AS debut_or_finale FROM finales
                ORDER BY release_date DESC;""")
connection.commit()

In [16]:
connection.close()